# Predicting Heart Disease

**Playground Series - Season 6 Episode 2**

https://www.kaggle.com/competitions/playground-series-s6e2

By Mike Johnson
****

## 1.0 Set Up

In [1]:
# Import libraries

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model building
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Gradient boosting
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Metrics (ROC AUC is your competition metric)
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve

# Hyperparameter tuning
import optuna

In [4]:
# Load data
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')

print(f"Training Set Shape: {train.shape}")
print(f"Test Set Shape: {test.shape}")

Training Set Shape: (630000, 14)
Test Set Shape: (270000, 13)


In [7]:
# View training data structure
train.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
id,,,,,,,,,,,,,,
0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence
